# Autograd: automatic differentiation

The ``autograd`` package provides automatic differentiation for all operations
on Tensors. It is a define-by-run framework, which means that your backprop is
defined by how your code is run, and that every single iteration can be
different.

In [1]:
import torch

Create a tensor:

In [112]:
# Create a 2x2 tensor with gradient-accumulation capabilities
x = torch.tensor([[1, 2], [3, 4]], requires_grad=True, dtype=torch.float32)
print(x)

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)


Do an operation on the tensor:

In [113]:
# Deduct 2 from all elements
y = x - 2
print(y)

tensor([[-1.,  0.],
        [ 1.,  2.]], grad_fn=<SubBackward0>)


``y`` was created as a result of an operation, so it has a ``grad_fn``.



In [6]:
print(y.grad_fn)

In [7]:
# What's happening here?
print(x.grad_fn)

None


In [8]:
# Let's dig further...
y.grad_fn

In [16]:
y.grad_fn.next_functions[0]

(<AccumulateGrad at 0x7f43b5e27fd0>, 0)

In [15]:
y.grad_fn.next_functions[0][0].variable

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)

In [33]:
# Do more operations on y
z = y * y * 3
a = z.mean()  # average

print(z)
print(a)

tensor([[ 3.,  0.],
        [ 3., 12.]], grad_fn=<MulBackward0>)
tensor(4.5000, grad_fn=<MeanBackward0>)


In [61]:
y.grad_fn.next_functions

((<AccumulateGrad at 0x7f43b5e27fd0>, 0), (None, 0))

In [62]:
y.grad_fn

In [63]:
z.grad_fn.next_functions

((<MulBackward0 at 0x7f43b5d2a7f0>, 0), (None, 0))

In [55]:
z.grad_fn

In [54]:
a.grad_fn.next_functions

((<MulBackward0 at 0x7f43b5d2a5e0>, 0),)

In [53]:
a.grad_fn

In [18]:
# Let's visualise the computational graph! (thks @szagoruyko)
from torchviz import make_dot

In [36]:
make_dot(a)

## Gradients

Let's backprop now `out.backward()` is equivalent to doing `out.backward(torch.tensor([1.0]))`

In [64]:
# Backprop
a.backward()

/home/pszczerbiak/.conda/envs/pDL/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729096996/work/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


Print gradients $\frac{\text{d}a}{\text{d}x}$.




In [65]:
# Compute it by hand BEFORE executing this
print(x.grad)

tensor([[-1.5000,  0.0000],
        [ 1.5000,  3.0000]])


Solution:
$$\frac{\partial a}{\partial x_{ij}}={\rm [apply\; definition]}=\frac32 y_{ij} \equiv \frac32 x_{ij} - 3$$

You can do many crazy things with autograd!
> With Great *Flexibility* Comes Great Responsibility

In [110]:
# Dynamic graphs!
x = torch.randn(3, requires_grad=True)

y = x * 2
i = 0
while y.data.norm() < 1000:
    y = y * 2
    i += 1
print(y)

tensor([ 954.8104, -556.4733,  443.3047], grad_fn=<MulBackward0>)


In [111]:
# If we don't run backward on a scalar we need to specify the grad_output
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


In [96]:
# BEFORE executing this, can you tell what would you expect it to print?
print(i)

9


## Inference

In [103]:
# This variable decides the tensor's range below
n = 3

In [107]:
z

tensor(6., grad_fn=<DotBackward>)

In [104]:
# Both x and w that allows gradient accumulation
x = torch.arange(1., n + 1, requires_grad=True)
w = torch.ones(n, requires_grad=True)
z = w @ x
z.backward()
print(x.grad, w.grad, sep='\n')

tensor([1., 1., 1.])
tensor([1., 2., 3.])


In [108]:
# Only w that allows gradient accumulation
x = torch.arange(1., n + 1)
w = torch.ones(n, requires_grad=True)
z = w @ x
z.backward()
print(x.grad, w.grad, sep='\n')

None
tensor([1., 2., 3.])


In [109]:
x = torch.arange(1., n + 1)
w = torch.ones(n, requires_grad=True)

# Regardless of what you do in this context, all torch tensors will not have gradient accumulation
with torch.no_grad():
    z = w @ x

try:
    z.backward()  # PyTorch will throw an error here, since z has no grad accum.
except RuntimeError as e:
    print('RuntimeError!!! >:[')
    print(e)

RuntimeError!!! >:[
element 0 of tensors does not require grad and does not have a grad_fn


## More stuff

Documentation of the automatic differentiation package is at
http://pytorch.org/docs/autograd.